# DataUp

Proyecto del curso "Taller de Proyecto". Análisis de entrevistas utilizando procesamiento de voz y técnicas de análisis de texto

In [1]:
#Imports
import pandas as pd
import speech_recognition as sr #Speech Recognition
from os import path
import matplotlib.pyplot as plt
#Pre procesamiento
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer


## 1.- Procesamiento de Voz

## 2.- Análisis del Texto

### 2.1.- Pre-procesamiento

In [91]:
texto = "Un edificio gris, achaparrado, de sólo treinta y cuatro plantas. Encima de la entrada principal las palabras: Centro de Incubacion y Condicionamiento de la Central de Londres, y, en un escudo, la divisa del Estado Mundial: Comunidad, Identidad, Estabilidad. Dentro, la enorme sala de la planta baja se hallaba orientada hacia el Norte. Fría a pesar del verano que reinaba en el exterior y del calor tropical de la sala, una luz cruda se filtraba a través de las ventanas buscando ávidamente algún cuerpo amortajado o alguna figura pálida, producto de las disecciones académicas; sin encontrar más que el cristal, el níquel y la brillante porcelana de un laboratorio. Lo invernal respondía a lo invernal. Las batas de los trabajadores eran blancas, y éstos llevaban las manos enfundadas en guantes de goma de un color pálido, como de cadáver. La luz era helada, muerta, fantasmal. Sólo de los amarillos cilindros de los microscopios se lograba arrancar cierta calidad de vida, deslizándose a lo largo de los tubos y formando una dilatada procesión de trazos luminosos que seguían la larga perspectiva de las mesas de trabajo."
pandasText = pd.DataFrame([texto],columns = ['Texto'], index = [1])

#### 2.1.1.- Eliminacion de stopwords y ruido

Removing stopwords: Stop words include the large number of prepositions, pronouns, conjunctions etc in sentences. These words need to be removed before we analyse the text, so that the frequently used words are mainly the words relevant to the context and not common words used in the text.

In [92]:
#Lista de stop words en espanol. Util para elminaras del texto original.
stopWords = set(stopwords.words("spanish"))

In [105]:
def preProcesamiento(textoPandas,stopWords, deleteStopWords = 1, lemanizar= 0):
    #Funcion que realiza el preProcesamiento del texto.
    
    #Convertir a Pandas.
    text = textoPandas['Texto'][1]
    #Pasar a minusculas.
    text = text.lower()
    #Eliminar comas y simbolos.
    text = re.sub(r'[^\w\s]','',text)
    #Convertir a vector de palabras.
    text = text.split()
    #Eliminar stopWords
    text = [word for word in text if not word in stopWords]
    #Decidir si lemanizar o no
    if lemanizar == 1:
        lem = WordNEtLemmatizer()
        text = [lem.lemmatize(word) for word in text]
    return text

In [110]:
#Generar el texto preProcesado. Se entrega un vector con las palabras clave.
text = preProcesamiento(pandasText,stopWords)

In [111]:
text

['edificio',
 'gris',
 'achaparrado',
 'sólo',
 'treinta',
 'cuatro',
 'plantas',
 'encima',
 'entrada',
 'principal',
 'palabras',
 'centro',
 'incubacion',
 'condicionamiento',
 'central',
 'londres',
 'escudo',
 'divisa',
 'mundial',
 'comunidad',
 'identidad',
 'estabilidad',
 'dentro',
 'enorme',
 'sala',
 'planta',
 'baja',
 'hallaba',
 'orientada',
 'hacia',
 'norte',
 'fría',
 'pesar',
 'verano',
 'reinaba',
 'exterior',
 'calor',
 'tropical',
 'sala',
 'luz',
 'cruda',
 'filtraba',
 'través',
 'ventanas',
 'buscando',
 'ávidamente',
 'algún',
 'cuerpo',
 'amortajado',
 'alguna',
 'figura',
 'pálida',
 'producto',
 'disecciones',
 'académicas',
 'encontrar',
 'cristal',
 'níquel',
 'brillante',
 'porcelana',
 'laboratorio',
 'invernal',
 'respondía',
 'invernal',
 'batas',
 'trabajadores',
 'blancas',
 'éstos',
 'llevaban',
 'manos',
 'enfundadas',
 'guantes',
 'goma',
 'color',
 'pálido',
 'cadáver',
 'luz',
 'helada',
 'muerta',
 'fantasmal',
 'sólo',
 'amarillos',
 'cilindro

In [96]:
for word in stopWords:
    if (word == 'y'):
        print('si')

si
